In [372]:
import pandas as pd
import numpy as np
import functools

In [373]:
labels = np.array([])
messages = np.array([])

with open('coursera_session_train.txt') as fp:
    train_data = fp.readlines()
    
with open('coursera_session_test.txt') as fp:
    test_data = fp.readlines()

In [374]:
frequencies_watched = {}
frequencies_buyed = {}

for line_ind, line in enumerate(train_data):
    watched, buyed = line[:-1].split(';')
    for ind, id in enumerate((watched.split(','))):
        if id in frequencies_watched:
            frequencies_watched[id][0] += 1
        else:
            frequencies_watched[id] = [1, (line_ind, ind)]
    
    for ind, id in enumerate(buyed.split(',')):
        if id in frequencies_buyed:
            frequencies_buyed[id][0] += 1
        else:
            frequencies_buyed[id] = [1, (line_ind, ind)]
      
    

In [375]:
def recomendation_on_watching_rate(recomendations_quantity, checked_goods):
    goods_frequency = []
    watched, buyed = checked_goods.split(';')
    for ind, id in enumerate(watched.split(',')):
        if id in frequencies_watched:

            if (id, frequencies_watched[id]) in goods_frequency:
                continue

            goods_frequency.append((id, frequencies_watched[id]))
        else:

            if (id, [0, [0, ind]]) in goods_frequency:
                continue

            goods_frequency.append((id, [0, [0, ind]]))
            
    def sort_func(a, b):
        if (a[1][0] > b[1][0]):
            return -1
        elif (a[1][0] < b[1][0]):
            return 1
        
        if (a[1][1][0] < b[1][1][0]):
            return -1
        elif (a[1][1][0] > b[1][1][0]):
            return 1
        
        if (a[1][1][1] < b[1][1][1]):
            return -1
        else:
            return 1
    
    recomendations = sorted(goods_frequency, key=functools.cmp_to_key(sort_func))
    recomendations = map(lambda x: x[0], recomendations)
    recomendations = list(dict.fromkeys(recomendations))
    return recomendations[0:recomendations_quantity]

In [376]:
def recomendation_on_buying_rate(recomendations_quantity, checked_goods):
    goods_frequency = []
    watched, buyed = checked_goods.split(';')
    for ind, id in enumerate(watched.split(',')):
        if id in frequencies_buyed:

            if (id, frequencies_buyed[id]) in goods_frequency:
                continue

            goods_frequency.append((id, frequencies_buyed[id]))
        else:

            if (id, [0, [0, ind]]) in goods_frequency:
                continue

            goods_frequency.append((id, [0, [0, ind]]))
            
    def sort_func(a, b):
        if (a[1][0] > b[1][0]):
            return -1
        elif (a[1][0] < b[1][0]):
            return 1
        
        if (a[1][1][0] < b[1][1][0]):
            return -1
        elif (a[1][1][0] > b[1][1][0]):
            return 1
        
        if (a[1][1][1] < b[1][1][1]):
            return -1
        else:
            return 1
    
    recomendations = sorted(goods_frequency, key=functools.cmp_to_key(sort_func))
    recomendations = map(lambda x: x[0], recomendations)
    recomendations = list(dict.fromkeys(recomendations))
    # print(sorted(goods_frequency, key=functools.cmp_to_key(sort_func)))
    return recomendations[0:recomendations_quantity]

In [377]:
def get_precision(num, recomended, buyed):
    numerator = 0
    for id in recomended:
        if id in buyed.split(','):
            numerator += 1
        
    return numerator / num


def get_recall(num, recomended, buyed):
    numerator = 0
    for id in recomended:
        if id in buyed.split(','):
            numerator +=1
            
    return numerator / len(buyed.split(','))

In [378]:
# Clean data from sessions where customer haven't made any purchase

train_data_cleaned = []
for line in train_data:
    if line.find(';') + 2 == len(line):
        continue
    
    train_data_cleaned.append(line)
    
    
test_data_cleaned = []
for line in test_data:
    if line.find(';') + 2 == len(line):
        continue
        
    test_data_cleaned.append(line)

In [379]:
def get_avg_recall_precision(data, recomend_func, recomendations_number):
    precisions = []
    recalls = []
    
    for line in data:
        recomendations = recomend_func(recomendations_number, line)
        precisions.append(get_precision(recomendations_number, recomendations, line[:-1].split(';')[1]))
        recalls.append(get_recall(recomendations_number, recomendations, line[:-1].split(';')[1]))
    
    precisions = np.array(precisions)
    recalls = np.array(recalls)
    
    return round(recalls.mean(), 2), round(precisions.mean(), 2)

In [380]:
# number of recomendations = 1, train_data_cleaned, wathcing recomendations
print('precision, recall (recomendations = 1, train_data_cleaned, wathcing recomendations):', get_avg_recall_precision(train_data_cleaned, recomendation_on_watching_rate, 1))

# number of recomendations = 5, train_data_cleaned, wathcing_recomendations
print('precision, recall (recomendations = 5, train_data_cleaned, wathcing recomendations):', get_avg_recall_precision(train_data_cleaned, recomendation_on_watching_rate, 5))

precision, recall (recomendations = 1, train_data_cleaned, wathcing recomendations): (0.44, 0.51)
precision, recall (recomendations = 5, train_data_cleaned, wathcing recomendations): (0.82, 0.21)


In [381]:
# number of recomendations = 1, test_data_cleaned, wathcing recomendations
print('precision, recall (recomendations = 1, test_data_cleaned, wathcing recomendations):', get_avg_recall_precision(test_data_cleaned, recomendation_on_watching_rate, 1))

# number of recomendations = 5, test_data_cleaned, wathcing_recomendations
print('precision, recall (recomendations = 5, test_data_cleaned, wathcing recomendations):', get_avg_recall_precision(test_data_cleaned, recomendation_on_watching_rate, 5))

precision, recall (recomendations = 1, test_data_cleaned, wathcing recomendations): (0.41, 0.48)
precision, recall (recomendations = 5, test_data_cleaned, wathcing recomendations): (0.8, 0.2)


In [382]:
# number of recomendations = 1, train_data_cleaned, buying recomendaions
print('precision, recall (recomendations = 1, train_data_cleaned, buying recomendaions):', get_avg_recall_precision(train_data_cleaned, recomendation_on_buying_rate, 1))

# number of recomendations = 1, train_data_cleaned, buying recomendations
print('precision, recall (recomendations = 5, train_data_cleaned, buying recomendaions):', get_avg_recall_precision(train_data_cleaned, recomendation_on_buying_rate, 5))

precision, recall (recomendations = 1, train_data_cleaned, buying recomendaions): (0.68, 0.79)
precision, recall (recomendations = 5, train_data_cleaned, buying recomendaions): (0.93, 0.25)


In [383]:
# number of recomendations = 1, test_data_cleaned, buying recomendations
print('precision, recall (recomendations = 1, test_data_cleaned, buying recomendaions):', get_avg_recall_precision(test_data_cleaned, recomendation_on_buying_rate, 1))

# number of recomendations = 1, test_data_cleaned, buying recomendations
print('precision, recall (recomendations = 5, test_data_cleaned, buying recomendaions):', get_avg_recall_precision(test_data_cleaned, recomendation_on_buying_rate, 5))

precision, recall (recomendations = 1, test_data_cleaned, buying recomendaions): (0.46, 0.53)
precision, recall (recomendations = 5, test_data_cleaned, buying recomendaions): (0.82, 0.21)
